In [11]:
import kiutils.symbol
import random
import os
import glob
import pandas as pd

# Reading symbol libraries from our BR symbols folder
# SYMBOLS_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-kicad-lib/Symbols"
SYMBOLS_PATH = r"C:/Users/JacobBrotmanKrass/Documents/Test Library"
JLC_PATH = r"C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/jlc-scraper/csv/Parts Inventory on JLCPCB.xlsx"
PARTS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/Kicad/Parts_Library.xlsx"


parts_sheet = pd.read_excel(PARTS_SHEET_PATH, index_col=[0])




In [13]:
for BR_ID, symbol in parts_sheet.iterrows():
    print(BR_ID, symbol.Name)

BRE-000000 C_0201_100nF_25V_X7R_10%
BRE-000001 C_0201_10nF_25V_X7R_10%
BRE-000002 C_0201_1nF_50V_C0G_1%
BRE-000003 C_0201_1uF_16V_X5R_10%
BRE-000004 C_0201_220nF_25V_X5R_10%
BRE-000005 C_0201_3p9F_50V_C0G_5%
BRE-000006 C_0201_470nF_6V3_X5R_10%
BRE-000007 C_0201_470pF_50V_C0G_5%
BRE-000008 C_0201_4n7F_25V_X7R_10%
BRE-000009 C_0201_6pF_50V_C0G_15%
BRE-000010 C_0402_100nF_50V_X7R_10%_AEC-Q200_mini
BRE-000011 C_0402_100pF
BRE-000012 C_0402_10nF_35V_X7R_2%
BRE-000013 C_0402_10nF_50V_X7R
BRE-000014 C_0402_10pF
BRE-000015 C_0402_10uF_10V
BRE-000016 C_0402_10uF_6V3_X5R
BRE-000017 C_0402_16pF
BRE-000018 C_0402_1nF_50V_NP0_5%
BRE-000019 C_0402_1uF_10V_X7R_10%
BRE-000020 C_0402_1uF_50V_X5R_20%
BRE-000021 C_0402_220nF_10V_X7R_10%
BRE-000022 C_0402_22nF_10V_X7R_10%
BRE-000023 C_0402_22pF
BRE-000024 C_0402_2u2F_10V_X7R_10%
BRE-000025 C_0402_330nF_6V3
BRE-000026 C_0402_33nF
BRE-000027 C_0402_3p9F
BRE-000028 C_0402_470nF_10V_X7R
BRE-000029 C_0402_470pF_50V_X7R
BRE-000030 C_0402_47nF_50V_X7R_10%_mini
B

In [14]:
os.chdir(SYMBOLS_PATH)
for lib_file in glob.glob("*.kicad_sym"):

    # Extract library nickname/category -- e.g., 0402_Capacitors
    lib_nickname = lib_file.replace(".kicad_sym", "")

    # Skip these libraries, they don't need to be documented (obsolete or not actual parts)
    if (lib_nickname == "BR~Deprecated") or (lib_nickname == "BR_Virtual_Parts"):
        continue
    print(lib_nickname)

    # Open symbol library
    lib_path = os.path.join(SYMBOLS_PATH, lib_file)
    symbol_lib = kiutils.symbol.SymbolLib().from_file(lib_path)
    
    # The Category is the library nickname, without the BR_ at the beginning
    category = lib_nickname[3:]

    break

In [ ]:
    # For each symbol in a given library, populate a new row in the Parts dataframe
    for symbol in symbol_lib.symbols:
        sheet_row = parts_sheet.loc['BR_ID', 

        # Symbol path in Kicad
        symbol_path = f"{lib_nickname}:{symbol.entryName}"

        ID_list.append(BR_ID)

        # Grab all the properties from the Kicad Symbol
        properties = {property.key: property.value for property in symbol.properties}

        # Remove a weird decoding error with the plus/minus sign
        bad_char = b'\xc3\x82'.decode()
        properties["Description"] = properties["Description"].replace(bad_char, '')

        # Some parts don't have a manufacturer and manufacturer part number -- deal with this some other time, for now just populate "None"
        if "Manufacturer" in properties:
            manufacturer = properties["Manufacturer"]
            mpn = properties["Manufacturer Part Num"]
        else:
            manufacturer = "None"
            mpn = "None"

        # Append a dictionary of all part properties to the parts list -- this will be converted to a Pandas dataframe at the end
        parts_list.append({"BR ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category})

        # Add BR ID to the symbol
        add_field_to_symbol(symbol_lib, symbol.libId, "BR ID", BR_ID)

        # Extract all supplier-related properties: supplier X with suppler number X
        supplier_properties = {property: properties[property] for property in properties if property[:8]=="Supplier"}
        supplier_numbers = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]=='P'}
        supplier_names = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]!='P'}

        # Ignore any thing that looks like this
        null_strings = ["", " ", "-", "--", "~", "NA", "N/A"]

        # Loop through and add vendors and the respective supplier number when the number X at the end matches (supplier 1 --> supplier part num 1)
        for name in supplier_names:
            for number in supplier_numbers:
                if name[-1] == number[-1]:
                    if supplier_numbers[number] not in null_strings:

                        # Append dictionary of supplier properties for each SPN to the vendors list
                        # this will be converted to Pandas dataframe at the end
                        vendors_list.append({"BR ID":BR_ID, "Supplier":supplier_names[name], "SPN":supplier_numbers[number], "Stock":0})

    symbol_lib.to_file()